## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

## Reading the data

In [2]:
bhbh = pd.read_csv('../Data/BHBH.csv')
bhbh['q'] = np.where(bhbh['Mass_1_out']>bhbh['Mass_0_out'], bhbh['Mass_0_out']/bhbh['Mass_1_out'], bhbh['Mass_1_out']/bhbh['Mass_0_out'] )
bhbh['q_label']=(bhbh['q'].round(2)*100).astype('int')
bhbh.head()

,Unnamed: 0,name,Mass_0,Z_0,spin_0,SN_0,Tstart_0,Mass_1,Z_1,a,...,Mass_1_out,Semimajor,Eccentricity,GWtime,BWorldtime,RLO,CE,tdelay,q,q_label
0,0,0_892687128228499,34.044,0.0001,0.0,rapid_gauNS,zams,26.514,0.0001,1850.0,...,10.136270,2132.70200,0.415168,1.063429e+11,6.826057,NaN,2.0,1.063429e+11,0.302760,30
1,1,0_853530625884132,72.066,0.0001,0.0,rapid_gauNS,zams,27.308,0.0001,2980.0,...,6.408674,77.27157,0.080601,5.383296e+05,6.673187,1.0,1.0,5.383363e+05,0.178708,18
2,2,0_663761388951870,45.129,0.0001,0.0,rapid_gauNS,zams,43.875,0.0001,8900.0,...,43.256140,7898.29400,0.869922,2.696385e+10,4.636688,NaN,NaN,2.696385e+10,0.972002,97
3,3,0_288577009013924,91.090,0.0001,0.0,rapid_gauNS,zams,53.664,0.0001,7730.0,...,53.322640,4714.46200,0.550149,1.109533e+11,4.094508,1.0,NaN,1.109533e+11,0.713893,71
4,4,0_736350695048425,53.827,0.0001,0.0,rapid_gauNS,zams,38.009,0.0001,15200.0,...,37.424980,15418.48000,0.019973,4.710726e+13,5.119633,NaN,NaN,4.710726e+13,0.704693,70


## Preparing the data for xgboost

In [3]:
data = bhbh[['Mass_0', 'Z_0', 'Mass_1', 'Z_1', 'a', 'e']][1:10000]
label = bhbh[['q_label']][1:10000]
dtrain = xgb.DMatrix(data, label=label)

In [12]:
data = bhbh[['Mass_0', 'Z_0', 'Mass_1', 'Z_1', 'a', 'e']][10001:11000]
label = bhbh[['q_label']][10001:11000]
dtest = xgb.DMatrix(data, label=label)

## Boost Parameters

In [13]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

param['eval_metric'] = ['auc', 'ams@0']

evallist = [(dtrain, 'train'), (dtest, 'eval')]

## Training

In [14]:
num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

/home/zenn/anaconda3/envs/dask/lib/python3.11/site-packages/xgboost/core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


XGBoostError: [23:12:29] ../src/objective/regression_obj.cu:148: label must be in [0,1] for logistic regression
Stack trace:
  [bt] (0) /home/zenn/anaconda3/envs/dask/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x674193) [0x7f7da1a74193]
  [bt] (1) /home/zenn/anaconda3/envs/dask/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x68d7b8) [0x7f7da1a8d7b8]
  [bt] (2) /home/zenn/anaconda3/envs/dask/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x2e0492) [0x7f7da16e0492]
  [bt] (3) /home/zenn/anaconda3/envs/dask/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x70) [0x7f7da153c5f0]
  [bt] (4) /home/zenn/anaconda3/envs/dask/lib/python3.11/lib-dynload/../../libffi.so.8(+0xa052) [0x7f7e0fbba052]
  [bt] (5) /home/zenn/anaconda3/envs/dask/lib/python3.11/lib-dynload/../../libffi.so.8(+0x88cd) [0x7f7e0fbb88cd]
  [bt] (6) /home/zenn/anaconda3/envs/dask/lib/python3.11/lib-dynload/_ctypes.cpython-311-x86_64-linux-gnu.so(+0x92e4) [0x7f7e0fbca2

## Saving the model

In [15]:
bst.save_model('0001.model')

NameError: name 'bst' is not defined

## Loading a saved model

In [16]:
bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model('0001.model')  # load data

## Predictions of the model

In [17]:
ypred = bst.predict(dtest)

In [18]:
ypred

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.